# Pré processamento

In [104]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

pd.options.mode.chained_assignment = None

In [150]:
#Carregando dataset original

columns = ['datetime', 'city', 'state', 'country', 'shape', 'duration_s', 'hours', 'comments', 'date_p', 'latitude', 'longitude']
df = pd.read_csv("completo.csv", header=0, names=columns, error_bad_lines=False, warn_bad_lines=False, low_memory=False)
df.head()

,datetime,city,state,country,shape,duration_s,hours,comments,date_p,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [151]:
#Trabsformando a coluna latitude e longitude em valores numéricos

df['latitude'] = df['latitude'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df['longitude'] = df['longitude'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

## Tratamento country = NaN

In [152]:
#separando o dataset em dois dataframes
#um contém os registros que possuem a coluna 'country' NaN
#o outro contém os registro que possuem a coluna 'country' preenchida

df_countrys_na = df[df['country'].isna()]
df = df[df['country'].notna()]

In [153]:
#Iniciando objetos geopy para preencher valores NaN da coluna 'Country' utilizando a latitude e longitude

geolocator = Nominatim(user_agent="geoapiExercises")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

In [154]:
#formatando latitude e longitude para serem utilizadas no Geopy

df_lat_lon = df_countrys_na[['latitude', 'longitude']]

for index, row in df_lat_lon.iterrows():
    df_lat_lon.at[index, 'location'] = str(row[0]) + ', ' + str(row[1])

df_lat_lon.drop('latitude', axis=1, inplace=True)
df_lat_lon.drop('longitude', axis=1, inplace=True)
df_lat_lon.to_csv('c_lat_lon.csv')

df_lat_lon

,location
1,"29.384209999999996, -98.581082"
18,"32.364167, -64.678611"
19,"0.0, 0.0"
30,"53.970571, -111.689885"
36,"-38.662334, 178.017649"
...,...
88623,"0.0, 0.0"
88659,"39.078889000000004, -78.427222"
88663,"40.858433000000005, -74.16375500000001"
88666,"50.465843, 22.891814"


In [43]:
#lendo o csv das latitudes e longitudes onde a coluna 'Country' apresentava valores NaN, em chunks

chunker = pd.read_csv('c_lat_lon.csv', header=0, error_bad_lines=False, warn_bad_lines=False, low_memory=False, chunksize=250)


In [44]:
#aplicando o Geopy para obter os países através das latitudes e longitudes

for piece in chunker:
    
    address = [reverse(loc) for loc in piece['location']]
    
    c = []
    for i in range(len(address)):
        try:
            c.append(address[i].raw['address']['country_code'])
        except:
            c.append('NULL')
    
    piece['country'] = c
    piece.to_csv('c_pieces.csv', mode = 'a', header = False)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('26.092222, -98.277778',), **{}).
Traceback (most recent call last):
  File "C:\Users\rober\anaconda3\lib\site-packages\urllib3\connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\rober\anaconda3\lib\site-packages\urllib3\util\connection.py", line 84, in create_connection
    raise err
  File "C:\Users\rober\anaconda3\lib\site-packages\urllib3\util\connection.py", line 74, in create_connection
    sock.connect(sa)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\rober\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\rober\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 381, in _make_request
    self._validate_conn(conn)
  File "C:\Users\rober\anaconda3\lib\site-packages\urllib3\co

RateLimiter caught an error, retrying (0/2 tries). Called with (*('34.033344, -118.037011',), **{}).
Traceback (most recent call last):
  File "C:\Users\rober\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 381, in _make_request
    self._validate_conn(conn)
  File "C:\Users\rober\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 978, in _validate_conn
    conn.connect()
  File "C:\Users\rober\anaconda3\lib\site-packages\urllib3\connection.py", line 362, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\rober\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 386, in ssl_wrap_socket
    return context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\rober\anaconda3\lib\ssl.py", line 500, in wrap_socket
    return self.sslsocket_class._create(
  File "C:\Users\rober\anaconda3\lib\ssl.py", line 1040, in _create
    self.do_handshake()
  File "C:\Users\rober\anaconda3\lib\ssl.py", line 1309, in do_handshake
    self._sslobj.do_han

ValueError: Must be a coordinate pair or Point

In [155]:
#formatando os resultados do Geopy

df_geopy = pd.read_csv('c_pieces.csv', header=None)
df_geopy.rename(columns={1:'index', 2:'lat_lon', 3: 'country'}, inplace=True)
df_geopy.set_index('index', inplace=True)
df_geopy.drop(0, axis=1, inplace=True)
df_geopy

,lat_lon,country
index,,
1,"29.384209999999996, -98.581082",us
18,"32.364167, -64.678611",bm
19,"0.0, 0.0",NaN
30,"53.970571, -111.689885",ca
36,"-38.662334, 178.017649",nz
...,...,...
47966,"41.877528999999996, -88.067012",us
47970,"64.282327, -135.0",ca
47979,"54.011022, -1.47102",gb


In [156]:
#preenchendo a coluna "Country" com os resultados do Geopy
#o modulo Geopy não foi capaz de determinar o país para todas as entradas (em torno de 12 mil), conseguindo apenas 6 mil
#ainda assim agregando mais informações ao dataset

df_countrys_na = df_countrys_na.iloc[:6499]
df_countrys_na['country'] = df_geopy['country']
#df_pieces_nan.to_csv('c_pieces_6000.csv')
df_countrys_na = df_countrys_na[df_countrys_na['country'].notna()]
df_countrys_na

,datetime,city,state,country,shape,duration_s,hours,comments,date_p,latitude,longitude
1,10/10/1949 21:00,lackland afb,tx,us,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082
18,10/10/1973 23:00,bermuda nas,NaN,bm,light,20,20 sec.,saw fast moving blip on the radar scope thin w...,1/11/2002,32.364167,-64.678611
30,10/10/1979 22:00,saddle lake (canada),ab,ca,triangle,270,4.5 or more min.,Lights far above&#44 that glance; then flee f...,1/19/2005,53.970571,-111.689885
36,10/10/1982 07:00,gisborne (new zealand),NaN,nz,disk,120,2min,gisborne nz 1982 wainui beach to sponge bay,1/11/2002,-38.662334,178.017649
41,10/10/1986 20:00,holmes/pawling,ny,us,chevron,180,3 minutes,Football Field Sized Chevron with bright white...,10/8/2007,41.523427,-73.646795
...,...,...,...,...,...,...,...,...,...,...,...
47959,5/20/1999 22:50,newcastle-upon-tyne (uk/england),NaN,gb,other,0,22:52pm,During the night of 20-05-99&#44 myself and a ...,6/23/1999,54.978252,-1.617780
47966,5/20/2000 15:00,glenn ellyn,il,us,other,30,30 seconds,circular shadow on ground reveals nothing in t...,11/20/2001,41.877529,-88.067012
47970,5/20/2000 21:50,yukon (canada),yk,ca,circle,180,2 -3 minutes,My husband and I was walking outside and we lo...,6/21/2000,64.282327,-135.000000
47979,5/20/2001 18:00,knaresborough&#44 north yorkshire (uk/england),NaN,gb,formation,300,3-5 minutes,there was 2 redish orange glowing spheres spin...,9/17/2003,54.011022,-1.471020


In [157]:
#unindo o df com os resultados do Geopy ao df original
df = pd.concat([df, df_countrys_na])
df

,datetime,city,state,country,shape,duration_s,hours,comments,date_p,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611
5,10/10/1961 19:00,bristol,tn,us,sphere,300,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.595000,-82.188889
...,...,...,...,...,...,...,...,...,...,...,...
47959,5/20/1999 22:50,newcastle-upon-tyne (uk/england),NaN,gb,other,0,22:52pm,During the night of 20-05-99&#44 myself and a ...,6/23/1999,54.978252,-1.617780
47966,5/20/2000 15:00,glenn ellyn,il,us,other,30,30 seconds,circular shadow on ground reveals nothing in t...,11/20/2001,41.877529,-88.067012
47970,5/20/2000 21:50,yukon (canada),yk,ca,circle,180,2 -3 minutes,My husband and I was walking outside and we lo...,6/21/2000,64.282327,-135.000000
47979,5/20/2001 18:00,knaresborough&#44 north yorkshire (uk/england),NaN,gb,formation,300,3-5 minutes,there was 2 redish orange glowing spheres spin...,9/17/2003,54.011022,-1.471020


In [158]:
#dropando colunas que não serão utilizadas

df.drop('hours', inplace=True, axis=1)
df.drop('date_p', inplace=True, axis=1)

In [159]:
#transformando colunas que são números do tipo object em numéricas. Os valores que estiver errados, são transformandos em NaN.

df['duration_s'] = df['duration_s'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df['latitude'] = df['latitude'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df['longitude'] = df['longitude'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

In [160]:
#dropando linhas que possuem algum valor NaN

df = df[df['latitude'].notna() & df['longitude'].notna() & df['duration_s'].notna()]

In [161]:
#separando date, e formatando para o formato datetime

df['datetime'] = df['datetime'].apply(lambda x: x.split(' '))
df['date'] = df['datetime'].apply(lambda x:x[0])
df['date'] =  pd.to_datetime(df['date'], format='%m/%d/%Y')
df.drop('datetime', axis=1, inplace=True)
df.head()

,city,state,country,shape,duration_s,comments,latitude,longitude,date
0,san marcos,tx,us,cylinder,2700.0,This event took place in early fall around 194...,29.883056,-97.941111,1949-10-10
2,chester (uk/england),NaN,gb,circle,20.0,Green/Orange circular disc over Chester&#44 En...,53.200000,-2.916667,1955-10-10
3,edna,tx,us,circle,20.0,My older brother and twin sister were leaving ...,28.978333,-96.645833,1956-10-10
4,kaneohe,hi,us,light,900.0,AS a Marine 1st Lt. flying an FJ4B fighter/att...,21.418056,-157.803611,1960-10-10
5,bristol,tn,us,sphere,300.0,My father is now 89 my brother 52 the girl wit...,36.595000,-82.188889,1961-10-10


In [162]:
df.dtypes

city                  object
state                 object
country               object
shape                 object
duration_s           float64
comments              object
latitude             float64
longitude            float64
date          datetime64[ns]
dtype: object

In [163]:
#verificando quais os shapes existentes

shapes = df['shape'].groupby(df['shape']).count()
shapes

shape
changed          1
changing      1993
chevron        957
cigar         2046
circle        7832
cone           340
crescent         2
cross          254
cylinder      1284
delta            8
diamond       1213
disk          5392
dome             1
egg            766
fireball      6137
flare            1
flash         1362
formation     2473
hexagon          1
light        16588
other         5759
oval          3801
pyramid          1
rectangle     1305
round            2
sphere        5270
teardrop       758
triangle      7974
unknown       5867
Name: shape, dtype: int64

In [164]:
#agrupando descrição de formas dos UFOs as quais julgamos serem pertinentes a uma única classe

df['shape'] = df['shape'].replace(to_replace ='changed', value = 'changing')
df['shape'] = df['shape'].replace(to_replace ='other', value = 'unknown')
df['shape'] = df['shape'].replace(to_replace ='delta', value = 'triangle')
df['shape'] = df['shape'].replace(to_replace ='crescent', value = 'unknown')
df['shape'] = df['shape'].replace(to_replace ='round', value = 'circle')
df['shape'] = df['shape'].replace(to_replace ='egg', value = 'oval')
df['shape'] = df['shape'].replace(to_replace ='flare', value = 'light')

In [165]:
#transformando a coluna duration_s de segundos para minutos

df['duration_s'] = df['duration_s'].apply(lambda x: x/60)
df.rename(columns={'duration_s': 'duration_m'}, inplace=True)

In [166]:
df.head()

,city,state,country,shape,duration_m,comments,latitude,longitude,date
0,san marcos,tx,us,cylinder,45.000000,This event took place in early fall around 194...,29.883056,-97.941111,1949-10-10
2,chester (uk/england),NaN,gb,circle,0.333333,Green/Orange circular disc over Chester&#44 En...,53.200000,-2.916667,1955-10-10
3,edna,tx,us,circle,0.333333,My older brother and twin sister were leaving ...,28.978333,-96.645833,1956-10-10
4,kaneohe,hi,us,light,15.000000,AS a Marine 1st Lt. flying an FJ4B fighter/att...,21.418056,-157.803611,1960-10-10
5,bristol,tn,us,sphere,5.000000,My father is now 89 my brother 52 the girl wit...,36.595000,-82.188889,1961-10-10


In [167]:
#exportando o df pre processado para um csv, para posterior utilização

df.to_csv('pre_processado_completo.csv', index=None)